In [1]:
import pandas as pd
import geopandas as gpd
import folium
import numpy as np
from glob import glob
from tqdm.notebook import tqdm

### Download SHAPE Files from OpenData

In [ ]:
import os
import urllib.request
import zipfile

urls = [
    "https://download1.bayernwolke.de/a/tn/lkr/tn_09162.zip", 
    "https://download1.bayernwolke.de/a/tn/lkr/tn_09184.zip",
    "https://download1.bayernwolke.de/a/tn/lkr/tn_09175.zip",
    "https://download1.bayernwolke.de/a/tn/lkr/tn_09177.zip",
    "https://download1.bayernwolke.de/a/tn/lkr/tn_09178.zip",
    "https://download1.bayernwolke.de/a/tn/lkr/tn_09186.zip", 
    "https://download1.bayernwolke.de/a/tn/lkr/tn_09174.zip",
    "https://download1.bayernwolke.de/a/tn/lkr/tn_09179.zip", 
    "https://download1.bayernwolke.de/a/tn/lkr/tn_09188.zip", 
]

base_path = '/Users/maltegenschow/Documents/Uni/SoSe23/Data Science Project/data/Nutzungsdaten/'

def download_and_unzip(urls, folder_path):
    # Create the folder if it doesn't exist
    os.makedirs(folder_path, exist_ok=True)

    for url in urls:
        # Extract the filename from the URL
        filename = url.split("/")[-1]
        file_path = os.path.join(folder_path, filename)

        # Download the file
        print(f"Downloading {filename}...")
        urllib.request.urlretrieve(url, file_path)

        #Create a folder with the same name as the zip file
        zip_folder = os.path.join(folder_path, os.path.splitext(filename)[0])
        os.makedirs(zip_folder, exist_ok=True)

        # Unzip the file into the created folder
        print(f"Unzipping {filename}...")
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(zip_folder)

        # Remove the downloaded zip file
        os.remove(file_path)

download_and_unzip(urls, base_path)

### Read in Data

In [2]:
# Read in all the files
dfs = []
base_path = '/Users/maltegenschow/Documents/Uni/SoSe23/Data Science Project/data/Nutzungsdaten/'
for file in tqdm(glob(base_path + '*/'), leave=False):
    dfs.append(gpd.read_file(file))

  0%|          | 0/9 [00:00<?, ?it/s]

In [3]:
# Concatenate all the files into one dataframe
df = gpd.GeoDataFrame(pd.concat(dfs, ignore_index=True))
# Convert the geometry column to the correct CRS
df['geometry_4326'] = df['geometry'].to_crs(epsg=4326)
df

,gml_id,oid,aktualit,nutzart,bez,name,geometry,geometry_4326
0,DEBYvAAAAAAO0JHXTN,DEBYvAAAAAAO0JHXTN,2021-03-30Z,Bahnverkehr,NaN,NaN,"POLYGON ((692157.370 5351040.150, 692166.340 5...","POLYGON ((11.59031 48.28337, 11.59043 48.28342..."
1,DEBYvAAAAAAGb9tuTN,DEBYvAAAAAAGb9tuTN,2014-11-25Z,Bahnverkehr,NaN,NaN,"POLYGON ((691799.730 5351745.450, 691801.590 5...","POLYGON ((11.58581 48.28982, 11.58584 48.28984..."
2,DEBYvAAAAABOhz9bTN,DEBYvAAAAABOhz9bTN,2020-10-05Z,Bahnverkehr,NaN,NaN,"POLYGON ((691704.730 5351823.300, 691716.450 5...","POLYGON ((11.58457 48.29055, 11.58473 48.29063..."
3,DEBYvAAAAABOhz6sTN,DEBYvAAAAABOhz6sTN,2017-08-29Z,Bahnverkehr,NaN,NaN,"POLYGON ((694317.060 5353466.820, 694391.290 5...","POLYGON ((11.62051 48.30452, 11.62153 48.30491..."
4,DEBYvAAAAAAGcAEVTN,DEBYvAAAAAAGcAEVTN,2021-03-30Z,Bahnverkehr,NaN,NaN,"POLYGON ((692071.160 5351303.380, 692076.730 5...","POLYGON ((11.58927 48.28577, 11.58934 48.28580..."
...,...,...,...,...,...,...,...,...
466844,DEBYvAAAAAAGjZL1TN,DEBYvAAAAAAGjZL1TN,2023-05-12Z,Gehölz,NaN,NaN,"POLYGON ((675021.920 5347536.540, 675051.440 5...","POLYGON ((11.35808 48.25685, 11.35847 48.25660..."
466845,DEBYvAAAAAAGbpZ0TN,DEBYvAAAAAAGbpZ0TN,2023-05-12Z,Straßenverkehr,NaN,Am Riedlsberg,"POLYGON ((675274.480 5347453.500, 675295.170 5...","POLYGON ((11.36145 48.25603, 11.36173 48.25602..."
466846,DEBYvAAAAAAGZ4v3TN,DEBYvAAAAAAGZ4v3TN,2023-05-12Z,Weg,NaN,NaN,"POLYGON ((675051.440 5347509.290, 675055.610 5...","POLYGON ((11.35847 48.25660, 11.35853 48.25662..."
466847,DEBYvAAAAAAGRXbkTN,DEBYvAAAAAAGRXbkTN,2023-05-12Z,Wohnbaufläche,NaN,NaN,"POLYGON ((675327.720 5347514.070, 675357.290 5...","POLYGON ((11.36219 48.25656, 11.36259 48.25653..."


### Check Data

In [ ]:
df.nutzart.unique()

In [ ]:
df.bez.nunique()

### Relabel to our use case

In [4]:
df['Label'] = ''
df['Label'] = np.where(df.nutzart.isin(['Fließgewässer', 'Stehendes Gewässer', 'Hafenbecken', 'Schiffsverkehr']), 'Wasser', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Wald', 'Forstwirtschaft']), 'Wald', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Wohnbaufläche']), 'Wohnbaufläche', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Straßenverkehr', 'Weg']), 'Verkehrsfläche/Weg', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Landwirtschaft']), 'Landwirtschaft', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Industrie- und Gewerbefläche']), 'Gewerbe', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Bahnverkehr', 'Flugverkehr']), 'Bahn/Flughafen', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Sport-, Freizeit- und Erholungsfläche']), 'Sport/Freizeit', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Fläche besonderer funktionaler Prägung']), 'Fläche besonderer funktionaler Prägung', df['Label'])
df['Label'] = np.where(df.nutzart.isin(['Fläche gemischter Nutzung']), 'Fläche gemischter Nutzung', df['Label'])

#### Subset only to Inner City for Plotting

In [12]:
from shapely.geometry import box
bbox = box(*[11.483459,48.056972,11.704559,48.196303])
df_subset = df[df.geometry_4326.intersects(bbox)]

### Plot some data

In [15]:
m = folium.Map(location  = [df.iloc[0,:].geometry_4326.centroid.y, df.iloc[0,:].geometry_4326.centroid.x], zoom_start = 12)
style_options = {
    'fillColor': '#0000ff',      # Fill color in hexadecimal format
    'color': '#0000ff',          # Line color in hexadecimal format
    'weight': 2,                 # Line weight in pixels
    'fillOpacity': 0.2           # Fill opacity (0.0 transparent to 1.0 opaque)
}
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True).add_to(m)

for label in tqdm(df_subset.Label.unique(), leave = False):
    folium.GeoJson(df_subset[df_subset.Label == label].geometry_4326, style_function=lambda x:style_options, name = label).add_to(m)
#folium.GeoJson(df[df.nutzart == 'Wald'].geometry_4326, style_function=lambda x:style_options, name = 'Water').add_to(m)
folium.LayerControl().add_to(m)
#m
m.save("Nutzungsdaten_Exploration.html")

  0%|          | 0/11 [00:00<?, ?it/s]